# SanityChecker: 

### Patch Clamp Electrophysiology Notebook that implements low level methods from PatchAnalyzer to do patch analysis without a GUI.



In [20]:
# ------------------------------------------------------------------
# Make the *outer* repo folder importable
# ------------------------------------------------------------------
import sys
from pathlib import Path
import pandas as pd

nb_path   = Path.cwd()                 # e.g. …/ProjectRoot/PatchAnalyzer/AnalysisTesting
repo_root = nb_path.parents[1]         # go up two levels → …/ProjectRoot/

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# sanity check (optional)
# print("repo root on sys.path:", repo_root)


In [21]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as W
from pathlib import Path

# ------------------------------------------------------------------
# PatchAnalyzer helpers
# ------------------------------------------------------------------
from PatchAnalyzer.models.data_loader import load_metadata
from PatchAnalyzer.models.ephys_loader import (
    load_voltage_traces_for_indices,
    load_current_traces,          # nested CC helper
)
from PatchAnalyzer.utils.passives import (
    compute_passive_params,       # VC
    compute_cc_passive_params,    # CC
)
from PatchAnalyzer.utils.spike_params import (
    Sweep,
    calc_firing_curve,
    calc_spike_metrics,
)

plt.rcParams["figure.dpi"] = 110


In [22]:
# Select multiple experiment dirs the same way the GUI’s WelcomePage does
exp_dirs = [
    Path(r"C:\Users\sa-forest\Documents\GitHub\PatchAnalyzer\Data\Test\2025_06_25-12_09"),   # TODO – add as many as you like
    # Path("/path/to/another/experiment"),
]
exp_dirs


[WindowsPath('C:/Users/sa-forest/Documents/GitHub/PatchAnalyzer/Data/Test/2025_06_25-12_09')]

In [23]:
# ---------------------------------------------------------------------------
# Build “cells” exactly like the GUI (unique by stage_x/y/z)
# ---------------------------------------------------------------------------
cells = {}   # { (exp_dir, (x,y,z)) : {"vc": list[traces], "cc": {current:trace}} }

# Load metadata for every experiment folder -------------------------------
meta_frames = []
for exp in exp_dirs:
    try:
        meta_frames.append(load_metadata([exp]))
    except ValueError:
        print(f"⚠  No CellMetadata in {exp}")

if not meta_frames:
    raise RuntimeError("No cell metadata loaded – check folder paths.")

meta_df = pd.concat(meta_frames, ignore_index=True)

# Detect which column holds the numeric sweep‑indices
idx_col = "indices" if "indices" in meta_df.columns else "index"
if idx_col not in meta_df.columns:
    raise RuntimeError("Neither 'indices' nor 'index' column found in metadata")

# Group by physical coordinates ------------------------------------------
for coord, sub in meta_df.groupby(["stage_x", "stage_y", "stage_z"]):
    exp_dir = Path(sub["src_dir"].iloc[0])
    sweep_indices = sub[idx_col].tolist()           # list of ints

    # ---- Voltage‑clamp traces
    vc_traces = []
    for idx in sweep_indices:
        for (t, cmd, rsp) in load_voltage_traces_for_indices(exp_dir, [idx]).values():
            vc_traces.append((t, cmd, rsp))

    # ---- Current‑clamp traces  {current_pA: (t,cmd,rsp)}
    cc_nested = {}
    cc_all = load_current_traces(exp_dir, sweep_indices)
    for per_cell_dict in cc_all.values():           # each is {current: trace}
        cc_nested.update(per_cell_dict)             # last duplicate wins

    cells[(exp_dir, coord)] = dict(vc=vc_traces, cc=cc_nested)

print(f"Found {len(cells)} unique physical cells")


2025-07-14 12:15:41,847 [INFO] Loaded 3 rows from C:\Users\sa-forest\Documents\GitHub\PatchAnalyzer\Data\Test\2025_06_25-12_09\CellMetadata\cell_metadata.csv
Found 1 unique physical cells


In [26]:
# print out dictionaries for sanity check
for (exp, coord), d in cells.items():
    print(f"{exp.name} {coord}: {len(d['vc'])} VC traces, {len(d['cc'])} CC traces")


2025_06_25-12_09 (np.float64(5968.358232524412), np.float64(51480.89419805703), np.float64(20338.2)): 3 VC traces, 25 CC traces


In [ ]:
rows = []

for (exp_dir, coord), data in cells.items():
    # ------------------------------------------------------------------
    # 1.  Voltage‑clamp passive ⟶ average across all VC sweeps
    ra = rm = cm = None
    if data["vc"]:
        ra_l, rm_l, cm_l = [], [], []
        for t, cmd, rsp in data["vc"]:
            out = compute_passive_params(t, cmd, rsp)
            if all(v is not None for v in out):
                ra_l.append(out[0]); rm_l.append(out[1]); cm_l.append(out[2])
        if ra_l:                                  # at least one good fit
            ra = np.mean(ra_l); rm = np.mean(rm_l); cm = np.mean(cm_l)

    # ------------------------------------------------------------------
    # 2.  Current‑clamp passive ⟶ compute on *every* sweep then average
    tau = rin = cap = rest = None
    cc_dict = data["cc"]                          # {current_pA: (t,cmd,rsp)}
    if cc_dict:
        tau_l, rin_l, cap_l, rest_l = [], [], [], []
        for t, cmd, rsp in cc_dict.values():
            pas = compute_cc_passive_params(t, cmd, rsp)
            if pas["membrane_tau_ms"] is not None:
                tau_l .append(pas["membrane_tau_ms"])
                rin_l .append(pas["input_resistance_MOhm"])
                cap_l .append(pas["membrane_capacitance_pF"])
                rest_l.append(pas["resting_potential_mV"])
        if tau_l:                                 # at least one good fit
            tau  = np.mean(tau_l)
            rin  = np.mean(rin_l)
            cap  = np.mean(cap_l)
            rest = np.mean(rest_l)

    # ------------------------------------------------------------------
    rows.append(dict(
        exp              = exp_dir.name,
        stage_x          = coord[0],
        stage_y          = coord[1],
        stage_z          = coord[2],
        Ra_MOhm          = ra,
        Rm_MOhm          = rm,
        Cm_pF            = cm,
        Rin_MOhm         = rin,
        tau_ms           = tau,
        Cm_CC_pF         = cap,
        Rest_mV          = rest,
    ))

df = (pd.DataFrame(rows)
        .sort_values(["exp", "stage_x", "stage_y", "stage_z"])
        .reset_index(drop=True))

df.head()




c:\Users\sa-forest\.conda\envs\patchanalyzer\Lib\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\sa-forest\.conda\envs\patchanalyzer\Lib\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
def plot_cell(exp_name, cell_id, clamp="VC"):
    rec = [k for k in cells if k[0].name == exp_name and k[1] == cell_id][0]
    data = cells[rec]

    if clamp == "VC":
        for t, cmd, rsp in data["vc"]:
            fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
            ax1.plot(t, cmd);  ax1.set_ylabel("Cmd (mV)")
            ax2.plot(t, rsp);  ax2.set_ylabel("Rsp (pA)")
            ax2.set_xlabel("Time (s)")
            fig.suptitle(f"{exp_name} – Cell {cell_id}  (VC)")
            plt.show()
    else:
        for I, (t, cmd, rsp) in sorted(data["cc"].items()):
            fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
            ax1.plot(t, cmd);  ax1.set_ylabel("Cmd (pA)")
            ax2.plot(t, rsp);  ax2.set_ylabel("Rsp (mV)")
            ax2.set_xlabel("Time (s)")
            fig.suptitle(f"{exp_name} – Cell {cell_id}  (CC {I:+.0f} pA)")
            plt.show()

# --- widgets ---
exp_w  = W.Dropdown(options=sorted({e.name for e, _ in cells}), description="Exp")
cell_w = W.IntText(value=1, description="Cell idx")
clamp_w= W.ToggleButtons(options=["VC", "CC"], description="Clamp")
ui = W.HBox([exp_w, cell_w, clamp_w])

out = W.interactive_output(plot_cell, {"exp_name": exp_w, "cell_id": cell_w, "clamp": clamp_w})
display(ui, out)


In [ ]:
fi_rows = []; spk_rows = []
for (exp, idx), data in cells.items():
    sweeps = []
    for I, (t, cmd, rsp) in data["cc"].items():
        sweeps.append(Sweep(time=t, command=cmd, response=rsp, sample_rate=1/(t[1]-t[0])))

    if not sweeps:
        continue

    fi = calc_firing_curve(sweeps)           # DataFrame
    fi["exp"] = exp.name; fi["cell_idx"] = idx
    fi_rows.append(fi)

    spk = calc_spike_metrics(sweeps)         # DataFrame
    spk["exp"] = exp.name; spk["cell_idx"] = idx
    spk_rows.append(spk)

fi_df   = pd.concat(fi_rows, ignore_index=True) if fi_rows else pd.DataFrame()
spk_df  = pd.concat(spk_rows, ignore_index=True) if spk_rows else pd.DataFrame()

fi_df.head(), spk_df.head()
